# Lab 13b: Evaluating the Performance of Minimum Variance Portfolios
Fill me in

## Setup
We set up the computational environment by including the `Include.jl` file. The `Include.jl` file loads external packages, various functions that we will use in the exercise, and custom types to model the components of our lab problem.

In [1]:
include("Include.jl");

    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/Desktop/julia_work/CHEME-5660-Labs-F23/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Labs-F23/Manifest.toml`
  Activating project at `~/Desktop/julia_work/CHEME-5660-Labs-F23`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Labs-F23/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Labs-F23/Manifest.toml`
    Updating registry at `~/.julia/registries/General.toml`
    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
   Installed Parsers ─ v2.8.0
   Installed PDMats ── v0.11.29
    Updating `~/Desktop/julia_work/CHEME-5660-Labs-F23/Project.toml`
  [b8f27783] ↑ MathOptInterface v1.21.0 ⇒ v1.22.0
    Updating `~/Desktop/julia_work/CHEME-5660-Labs-F23/Manifest.toml`
  [b8f27783] ↑ MathOptInterface v1.21.0 ⇒ v1.22.0
  [739be429

## Prerequisites: Load and clean the historical dataset
We gathered a daily open-high-low-close `dataset` for each firm in the [S&P500](https://en.wikipedia.org/wiki/S%26P_500) since `01-03-2018` until `10-31-2023`, along with data for a few exchange traded funds and volatility products during that time. In this block of code, we:
* Load and clean the historical data, we stored the clean data in the `dataset` variable.
* We calculate the expected excess return $\mathbb{E}(R_{i})$ for each `ticker` in `dataset`, however, we are only going to use the expected excess return for `SPY`, our benchmark portfolio
* We load the `single index models` estimated in `Lab 12c`, we store these models in the `sims` variable.
* We compute the expected excess return, and the covariance for each `ticker` in `dataset` using a `single index model`. These values are stored in the `μ_sim` and `Σ_sim` variables.

In [3]:
original_dataset = MyPortfolioDataSet() |> x-> x["dataset"];

### Clean the data
Not all of the tickers in our dataset have the maximum number of trading days for various reasons, e.g., acquistion or de-listing events. Let's collect only those tickers with the maximum number of tradition days.

* First, let's compute the number of records for a company that we know has a maximim value, e.g., `AAPL` and save that value in the `maximum_number_trading_days` variable:

In [4]:
maximum_number_trading_days = original_dataset["AAPL"] |> nrow;

In [6]:
dataset = Dict{String,DataFrame}();
for (ticker,data) ∈ original_dataset
    if (nrow(data) == maximum_number_trading_days)
        dataset[ticker] = data;
    end
end
dataset;

Let's get a list of firms that we have in cleaned up `dataset`, and save it ib the `all_tickers` array

In [7]:
all_tickers = keys(dataset) |> collect |> sort;

Finally, let's set some constant values, e.g., the `risk_free_rate`

In [11]:
risk_free_rate = 0.05;

### Compute the covariance and expected yield for all firms in the dataset
The expected return $\mathbb{E}(r_{i})$ and covariance matrix $\Sigma$ will be used in our calculations, so we'll provide values for both of these items for the entire data set (all `N = 459` tickers), and then you can pick out which tickers you are interested in. 

* First, we compute the expected (annualized) log return by passing the `dataset` and the entire list of firms we have in the dataset (held in the $N\times{1}$ `all_array` array) to the `log_return_matrix(...)` method. The result is stored in the `all_firms_return_matrix` variable, a $T-1\times{N}$ array of log return values. Each row of `all_firms_return_matrix` corresponds to a time-value, while each column corresponds to a firm:

In [13]:
all_firms_excess_return_matrix = log_return_matrix(dataset, all_tickers, 
    Δt = (1.0/252.0), risk_free_rate = risk_free_rate);

Next, we estimate the expected excess return for each firm from the `all_firms_excess_return_matrix` using the `mean(...)` function, which is exported by the [Statistics.jl package](https://docs.julialang.org/en/v1/stdlib/Statistics/). We set the $N\times{1}$ vector of expected values, i.e., $\mathbb{E}(R_{1}),\dotsc,\mathbb{E}(R_{N})$ in the variable $\mu$:

In [14]:
μ = mean(all_firms_excess_return_matrix, dims=1) |> vec;

### Build single index model returns and covariance

In [9]:
sims = load(joinpath(_PATH_TO_DATA, "SIM-SP500-01-03-18-to-10-31-23.jld2")) |> x->x["sim"];

Store the index of the market portfolio in the `index_SPY` variable, and then the expected return of the market portfolio $\mathbb{E}(R_{m})$ in the `R_SPY` vector:

In [15]:
index_SPY = findfirst(x->x=="SPY", all_tickers);
R_SPY = μ[index_SPY];

Now, compute the expected return for each ticker in `dataset` using the `single index models` computed in `Lab 12b` using a `for` loop. For iteration of the loop:
* Get the `ticker` symbol from the `all_tickers` array, and pull out the corresponding `single index model` from the `sims` dictionary
* Next, pull out the $(\alpha, \beta)$ parameters from the `sim` model instance and compute the expected return $\mathbb{E}(R_{i})$. Store the expected return in the `μ_sim` array

In [16]:
μ_sim = Array{Float64,1}();
for i ∈ eachindex(all_tickers)
    
    myticker = all_tickers[i];
    sim = sims[myticker];
    
    αᵢ = sim.α
    βᵢ = sim.β
    Rᵢ = αᵢ+βᵢ*R_SPY
    
    push!(μ_sim,Rᵢ)
end

Next let's build the `single index model` version of the covariance array. Substituting the single index model for the excess return of asset $i$ and $j$ into the covariance expression (and simplifying) gives:

$$
\begin{equation*}
\text{cov}(R_{i}, R_{j}) = \begin{cases}
\beta_{i}^{2}\sigma_{m}^{2}+\sigma_{\epsilon_{i}}^{2} & i = j \\
\beta_{i}\beta_{j}\sigma_{m}^2 & i \neq j
\end{cases}
\end{equation*}
$$

where $\sigma_{m}^2$ denotes the variance of the excess return of the market,  and $\sigma_{\epsilon_{i}}^{2}$ denotes the variance of the firm-specific error model.

In [21]:
σₘ = std(all_firms_excess_return_matrix[:, index_SPY]);

We populate the single index covariance array using nested `for` loops:

In [20]:
Σ_tmp = Array{Float64,2}(undef, length(μ), length(μ));
for i ∈ eachindex(all_tickers)
    outer_ticker = all_tickers[i];
    sim_outer = sims[outer_ticker];
    
    for j ∈ eachindex(all_tickers)
        
        inner_ticker = all_tickers[j];
        sim_inner = sims[inner_ticker];
        
        if (i == j)
            βᵢ = sim_outer.β
            ϵᵢ = sim_outer.ϵ
            σ_ϵᵢ = params(ϵᵢ)[2];
            Σ_tmp[i,j] = ((βᵢ)^2)*((σₘ)^2)+(σ_ϵᵢ)^2
        else
            βᵢ = sim_outer.β
            βⱼ = sim_inner.β
            Σ_tmp[i,j] = βᵢ*βⱼ*(σₘ)^2
        end
    end
end
Σ_sim  = Σ_tmp |> x-> x*(1/252);

## Task 1: Select tickers for your `risky-only` portfolio
Before we compute the efficient frontier, we must decide which stocks to include in the risky portfolio. One strategy is to pick stocks that have small or even negative covariance. Let's explore this idea. 

* Specify a collection of `ticker symbols`, e.g., `AAPL`, `MSFT`, etc., that interest you in the `my_list_of_tickers` array. We'll  put the indexes of your tickers in the `my_list_of_firm_ids` array:

In [22]:
my_list_of_tickers = ["AAPL", "INTC", "MSFT", "MU", "AMD", "GS", "BAC", "WFC", "C", "F"];
my_list_of_firm_ids = Array{Int64,1}();
for ticker ∈ my_list_of_tickers
    firm_index = findfirst(x->x==ticker, all_tickers);    
    push!(my_list_of_firm_ids, firm_index)
end